In [ ]:
def parse_easylist_selectors(file_path):
    selectors = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Only consider lines with CSS selector-based rules
            if line.startswith('##'):
                selector = line[2:].strip()
                selectors.append(selector)
    return selectors

parse_easylist_selectors("easylist.txt")

In [ ]:
import requests
from bs4 import BeautifulSoup

def check_ads(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Check for common ad-related elements or scripts
        ad_elements = soup.find_all(['iframe', 'script', 'div'], 
                                    id=lambda x: x and 'ad' in x.lower(),
                                    class_=lambda x: x and 'ad' in x.lower())

        return len(ad_elements) > 0
    except Exception as e:
        print(f"Error occurred: {e}")
        return False

# Example usage
url = 'https://www.espncricinfo.com/'
has_ads = check_ads(url)
print(f"Ads found: {has_ads}")


In [ ]:
import pandas as pd
import random


In [ ]:
df = pd.read_csv("/Users/kaleemullahqasim/Downloads/top-1m.csv")

In [ ]:
df

In [ ]:
filtered_df = df.iloc[150000:-150000]

# Randomly sample 1000 domains
random_domains = filtered_df['google.com'].sample(n=1000, random_state=random.seed()).tolist()


In [ ]:
random_domains

In [ ]:
formatted_domains = ['https://' + domain for domain in random_domains]

# Output the list
for domain in formatted_domains:
    print(domain)

In [ ]:
domains_data = pd.DataFrame(formatted_domains)

In [ ]:
domains_data.to_parquet('urls.parquet')

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import os

def download_ads(url, save_dir='./ads'):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Identify ad elements using CSS classes, attributes, or patterns
        ad_elements = soup.find_all(
            'div', class_=['my-ad', 'ad-container', 'adsense-unit']
        )  # Replace with appropriate selectors

        # Extract ad information and download images
        for ad_element in ad_elements:
            image_url = ad_element.find('img')['src']
            if image_url:
                # Download ad image
                filename = os.path.basename(image_url)
                save_path = os.path.join(save_dir, filename)

                try:
                    download_image(image_url, save_path)
                except Exception as e:
                    print(f"Error downloading image: {e}")
                    continue

def download_image(image_url, save_path, retry_attempts=3):
    for attempt in range(retry_attempts):
        try:
            image_response = requests.get(image_url)
            if image_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(image_response.content)
                break
        except Exception as e:
            print(f"Error downloading image: {e}")
            if attempt < retry_attempts - 1:
                print(f"Retrying attempt {attempt + 1}...")
                time.sleep(1)
            else:
                raise e

# Example usage
download_ads('https://www.espncricinfo.com/')


In [ ]:
import requests

In [ ]:
import requests

def download_image(image_url, save_path):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(save_path, 'wb') as file:
                file.write(response.content)
            print(f"Image successfully downloaded to {save_path}")
        else:
            print(f"Failed to download image: status code {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Request exception occurred: {e}")
    except IOError as e:
        print(f"IOError occurred: {e}")

if __name__ == "__main__":
    image_url = 'https://en.wikipedia.org/wiki/File:Google_Images_2015_logo.svg'
    save_path = '/Users/kaleemullahqasim/Desktop/Prof Xiu Hai Tao/data/google_logo.svg'

    download_image(image_url, save_path)


In [ ]:
import pandas as pd
import csv

In [ ]:
df = pd.read_parquet("urls.parquet")

In [ ]:
df.to_csv("urls.csv")

In [ ]:
%pip install --upgrade requests urllib3 certifi

In [1]:
import requests
from tqdm import tqdm
import threading

def download_file_with_progress(url, local_filename):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            with tqdm(total=total_size, unit='iB', unit_scale=True, desc=local_filename) as bar:
                with open(local_filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            bar.update(len(chunk))
            print(f"File downloaded successfully as {local_filename}")
        else:
            print(f"Error occurred: Status code {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

def download_files(urls):
    threads = []
    for url in urls:
        file_name = "/Users/kaleemullahqasim/Desktop/Prof Xiu Hai Tao/ads/" + url.split('/')[-1]
        thread = threading.Thread(target=download_file_with_progress, args=(url, file_name))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

urls = [
    # "https://storage.googleapis.com/ads-dataset/subfolder-1.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-2.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-3.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-4.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-5.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-6.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-7.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-8.zip",
    # "https://storage.googleapis.com/ads-dataset/subfolder-9.zip",
    "https://storage.googleapis.com/ads-dataset/subfolder-10.zip",
]

download_files(urls)


/Users/kaleemullahqasim/Desktop/Prof Xiu Hai Tao/ads/subfolder-10.zip: 100%|██████████| 6.89G/6.89G [04:07<00:00, 27.8MiB/s] 

File downloaded successfully as /Users/kaleemullahqasim/Desktop/Prof Xiu Hai Tao/ads/subfolder-10.zip
